## Prepare the inputs

- encoder_input_data: 2D array of shape `(num_images * 5, 512)`
- decoder_input_data: 3D array of shape `(num_captions, max_words_in_sentence, num_words)`
- decoder_output_data: same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]

## Image Preprocessing

obtain bottleneck features

In [97]:
import numpy as np

In [98]:
# Since there are 5 captions per image, duplicate the bottleneck features
def duplicate_bottleneck_features(features):
    num_captions = 5 # 5 stands for number of captions per image
    num_rows = features.shape[0] * num_captions 

    features_dup = np.zeros((num_rows, features.shape[1]))
    for i, image in enumerate(features):
        for j in range(num_captions):
            features_dup[i*num_captions + j] = image
    return features_dup    

In [99]:
bottleneck_features = np.load('bottleneck_features/Flicker8k_bottleneck_features_VGG16_avgpooling.npz')
bottleneck_features_train = bottleneck_features["train"]
bottleneck_features_validation = bottleneck_features["validation"]
bottleneck_features_test = bottleneck_features["test"]

bottleneck_features_train_dup = duplicate_bottleneck_features(bottleneck_features_train)
bottleneck_features_validation_dup = duplicate_bottleneck_features(bottleneck_features_validation)
bottleneck_features_test_dup = duplicate_bottleneck_features(bottleneck_features_test)

In [100]:
print(bottleneck_features_train_dup.shape)
print(bottleneck_features_validation_dup.shape)
print(bottleneck_features_test_dup.shape)

(30000, 512)
(5000, 512)
(5000, 512)


## Word Embedding

In [101]:
from caption_utils import *
train_fns_list, dev_fns_list, test_fns_list = load_split_lists()

train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)     
captions_data = (train_captions_raw.copy(), dev_captions_raw.copy(), test_captions_raw.copy())
train_captions, dev_captions, test_captions = process_captions(captions_data, token2idx)

In [102]:
print(len(vocab))

2531


## Caption Preprocessing


In [184]:
captions_onehot = np.load('preprocessed_captions/Flicker8k_onehot_'+str(len(vocab))+'_words.npz')
train_captions_onehot = captions_onehot["train"]
validation_captions_onehot = captions_onehot["validation"]
test_captions_onehot = captions_onehot["test"]
train_captions_onehot = train_captions_onehot.astype(int)
validation_captions_onehot = validation_captions_onehot.astype(int)
test_captions_onehot = test_captions_onehot.astype(int)

In [ ]:
print(train_captions_onehot.shape)
print(validation_captions_onehot.shape)
print(test_captions_onehot.shape)

In [185]:
print(train_captions_onehot.dtype, validation_captions_onehot.dtype, test_captions_onehot.dtype)

int64 int64 int64


In [186]:
def captions_onehot_split(captions_onehot):
    """ returns decoder input data and decoder target data """
    return captions_onehot[:, :-1, :], captions_onehot[:, :, :]

### Training captions - > decoder input, target data  

In [187]:
train_decoder_input, train_decoder_target = captions_onehot_split(train_captions_onehot)
validation_decoder_input, validation_decoder_target = captions_onehot_split(validation_captions_onehot)
test_decoder_input, test_decoder_target = captions_onehot_split(test_captions_onehot)

In [188]:
train_encoder_output = bottleneck_features_train_dup

In [189]:
print("Decoder Input", train_decoder_input.shape, "\nDecoder Target", train_decoder_target.shape)
print("Encoder Output", train_encoder_output.shape)

Decoder Input (30000, 38, 2531) 
Decoder Target (30000, 39, 2531)
Encoder Output (30000, 512)


## Build Model

In [190]:
batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 300  # Latent dimensionality of the encoding space.
num_samples = train_encoder_output.shape[0]

-------------------------
# Experiment 

## Parameters

In [191]:
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, BatchNormalization, RepeatVector, Concatenate, TimeDistributed
import keras.backend as K

In [192]:
emb_size = 150
lstm_size = 300
vocab_size = len(vocab)
max_length = train_decoder_target.shape[1]
print(max_length)

39


In [193]:
K.clear_session()

## Image Embedding

In [194]:
image_input = Input(shape=(train_encoder_output.shape[1], ))
img_emb = Dense(emb_size, activation='relu')(image_input)
img_emb = RepeatVector(1)(img_emb)
print(img_emb.shape)

(?, 1, 150)


## Word Embedding

In [195]:
caption_inputs = Input(shape=(max_length-1, ))
word_emb = Embedding(input_dim=vocab_size, output_dim=emb_size)(caption_inputs)
print(word_emb.shape)

(?, 38, 150)


In [196]:
seq_input = Concatenate(axis=1)([img_emb, word_emb])
print(seq_input.shape)

(?, 39, 150)


In [197]:
gru_cell = GRU(lstm_size, return_sequences=True)(seq_input)
seq_out = TimeDistributed(Dense(vocab_size, activation='softmax'))(gru_cell)
print(seq_out.shape)

(?, 39, 2531)


In [202]:
Model(inputs=[train_encoder_output, train_decoder_input],
              outputs=train_decoder_target)

TypeError: unhashable type: 'numpy.ndarray'

In [161]:
print(max_length)

37


# Remove lines below